# Challenge Prompting

Resolver los siguientes ejercicios dejando el codigo con su ejecucion.

Importar las librerias necesarias y **correr las celdas para visualizar el resultado en cada ejercicio**.

In [1]:
## bloque importacion de librerias

import json
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
## bloque variables de entorno
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file

api_key = os.getenv("COHERE_API_KEY")
print(api_key)  # Verify the key is loaded


BTa138QUrWAZ74XpT2d8t4FWYwFc4NF5mF0Z9x81


In [ ]:
## bloque conexion a Cohere
import cohere

co = cohere.ClientV2()
# alternativa:
# co = cohere.ClientV2(api_key)

response = co.chat(
    model="command-r-plus-08-2024",
    messages=[{"role": "user", "content": "hello world!"}],
)

print(response)



id='0ef7c213-cb0f-45b4-8640-4328bb780dd1' finish_reason='COMPLETE' prompt=None message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='Hello! How can I help you today?')], citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=3.0, output_tokens=9.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=204.0, output_tokens=9.0)) logprobs=None


## Ejercicio 1

Extraccion de entidades

Utilizar el LLM para extraer las siguientes entidades del texto medico.

- Paciente:
    - Nombre
    - Edad
- Fecha de admisión
- Síntomas
- Diagnóstico
- Tratamiento recomendado

**Aclaracion:** 

La salida tiene que ser un **string con formato de tipo json**, el cual se convertira en un diccionario de Python.

Si la linea de conversion en test da error el ejercicio no esta completo.

In [ ]:
# ejemplo 

# texto a analizar
"""La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""


# respuesta del LLM
{
  "paciente": {
    "nombre": "María González",
    "edad": 45
  },
  "fecha_admision": "2023-08-05",
  "sintomas": [
    "fatiga crónica",
    "dolores musculares"
  ],
  "diagnostico": "fibromialgia",
  "tratamiento": [
    "fisioterapia",
    "medicamentos analgésicos"
  ]
}

In [ ]:
#####

In [2]:
text_to_analize = """Sofía López, de 28 años, ingresó al Hospital Infantil el 3 de abril de 2023 debido a fiebre alta y tos persistente./
Después de varias pruebas, se le diagnosticó neumonía. La pediatra responsable, Dra. Claudia Torres, indicó tratamiento con antibióticos y reposo./
La próxima evaluación será el 10 de abril."""

In [44]:
# your code here
import cohere

co = cohere.ClientV2()


text_to_analize = """Sofía López, de 28 años, ingresó al Hospital Infantil el 3 de abril de 2023 debido a fiebre alta y tos persistente./
Después de varias pruebas, se le diagnosticó neumonía. La pediatra responsable, Dra. Claudia Torres, indicó tratamiento con antibióticos y reposo./
La próxima evaluación será el 10 de abril."""

responses = co.chat(
    model = "command",
    messages = [{"role":"system", "content":"Eres un asistente virtual medico, capaz de generar resumenes sobre informes medicos."},
                {"role":"user", "content":f"Sobre el informe {text_to_analize}, debes reconocer y ordenar los siguientes datos: datos personales del paciente (nombre y edad), su fecha de admision, sintomas, diagnostico y tratamiento. {text_to_analize}"}]
)
body = responses.message.content[0].text
#print(body)

resp_light = co.chat(
    model = "command-r-plus-08-2024",
    messages = [{"role":"system", "content":"Eres un asistente virtual especializado en conversion de datos en formato json, solo datos relevantes. Si vez algun campo con mas de un dato, generalo como una lista. Ademas, el contenido debe ser en el idioma español"},
                {"role":"user", "content":f"Convierte esta informacion en una respuesta json: {body}."}]
)

llm_response = resp_light.message.content[0].text
print(llm_response)

#test
final_result = json.loads(llm_response)

print(final_result)



{
  "paciente": {
    "nombre": "Sofia Lopez",
    "edad": "28 años"
  },
  "fecha_ingreso": "3 de abril de 2023",
  "sintomas": [
    "Fiebre alta",
    "Tos persistente"
  ],
  "diagnostico": "Neumonía",
  "tratamiento": [
    "Antibióticos",
    "Reposo"
  ],
  "fecha_seguimiento": "10 de abril de 2023"
}
{'paciente': {'nombre': 'Sofia Lopez', 'edad': '28 años'}, 'fecha_ingreso': '3 de abril de 2023', 'sintomas': ['Fiebre alta', 'Tos persistente'], 'diagnostico': 'Neumonía', 'tratamiento': ['Antibióticos', 'Reposo'], 'fecha_seguimiento': '10 de abril de 2023'}


## Ejercicio 2

Tenemos dos funciones en Python, una llamada *'add_contact'* y otra llamada *'get_information'*.

**Utilizar algun LLM que permita funtion calling** y desarrollar un codigo secuencial automatico que consiga:

Interpretar la consulta del usuario, identificar a que funcion llamar, luego llamarla (si es que aplica) y darle una respuesta final al usuario.  (usar function calling para esta solucion)

La entrada a dicho codigo es la consulta del usuario, a continuacion algunos ejemplos:

- "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."
- "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
- "Cual es el Email de Juan Pérez.?"

Salidas esperadas de dichos ejemplos (variaran porque las genera el LLM):
-  El contacto fue anadido con exito
-  Se anadio el contacto
-  El email de juan perez es juanperez@mail.com

Link de ayuda: https://github.com/cohere-ai/notebooks/blob/main/notebooks/agents/Vanilla_Tool_Use_v2.ipynb

In [112]:
import cohere

co = cohere.ClientV2()

def add_contact(name, phone, email):
    """
    Agrega un contacto al diccionario.
    Parámetros:
        name (str): Nombre del contacto.
        phone (str): Número de teléfono del contacto.
        email (str): Correo electrónico del contacto.
    Retorna:
        str: Mensaje confirmando la adición del contacto.
    """
    contacts[name] = {'phone': phone, 'email': email}
    return "Contacto añadido con éxito."

def get_information(name):
    """
    Recupera la información de un contacto.
    Parámetros:
        name (str): Nombre del contacto.
    Retorna:
        dict/str: Información del contacto o un mensaje si no existe.
    """
    if name in contacts:
        return contacts[name]
    else:
        return "Contacto no encontrado."
    
contacts = {
                        'Joaquin Lopez':{'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'},
                      'Flavio Oncativo':{'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}
}

aviable_fuctions = {
    "add_contact":add_contact,
    "get_information":get_information
}

tools = [
    {
        "type":"function",
        "function":{
            "name":"add_contact",
            "description":"Agregar los datos ingresados de una persona a la lista de contactos, agregando los datos nombre,telefono y email. Mostrar el mensaje que retorna",
            "parameters":{
                "type":"object",
                "properties":{
                    "name":{
                        "type":"string",
                        "description":"Nombre de la persona que se quiere registrar"
                    },
                    "phone":{
                        "type":"integer",
                        "description":"Numero de telefono de la persona que se quiere registrar"
                    },
                    "email":{
                        "type":"string",
                        "description":"Correo electronico o email de la persona que se quiere registrar"
                    }
                },
                "required":["name","phone","email"]
            }
        }
    },

    {
        "type":"function",
        "function":{
            "name":"get_information",
            "description":"Retorna los datos del nombre que este en la lista de contactos.",
            "parameters":{
                "type":"object",
                "properties":{
                    "name":{
                        "type":"string",
                        "description":"Nombre de la persona que se utiliza para la busqueda de sus datos en la lista de contactos"
                    }
                },
                "required":["name"]
            }
        }
    }
]

messages = [{"role":"system", "content":f"""Eres un asistente virtual. Tendras a disposicion funciones que te ayudaran a manejar solicitudes 
             de agregar o traer informacion de la persona que te indica. Cuando no te soliciten alguna de esas acciones puedes mantener una comunicacion 
             referido a la lista de contactos u otros temas"""},
            {"role":"user", "content":"Cual es el email de Flavio Oncativo?"}]

response_ = co.chat(
    model="command-r-plus",
    messages=messages,
    tools=tools
)

for t in response_.message.tool_calls:
    function_name = t.function.name
    function_argument = json.loads(t.function.arguments)
    tool_response = aviable_fuctions[function_name](**function_argument)


response_final = co.chat(
    model="command-r-plus",
    messages=[{"role":"system", "content":"""Eres un profesor de programacion, que es capaz de recibir cualquier tipo de dato o variable para manipularlos segun el prompt del usuario. 
               Debes ser capaz de leer prompts de usuarios e interpretar su intencion. Tu respuesta debe ser un mensaje sencillo que no supere las 20 palabras. 
               Recibiras la respuesta del modelo para que sigas la orientacion de su idea"""},
              {"role":"user","content":f"usuario prompt: {messages[1]['content']}, datos encontrados: {tool_response}"}]
    
)

print(response_final.message.content[0].text)


# La idea fue utilizar dos modelos porque el primero me solica indicar siempre que tipo de accion iba a utilizar,
# entonces pense en poner otro modelo que pueda tomar los datos que le brida el primer modelo, como el resultado de la funcion y el primer prompt del user que le pasan)
# y generar una corta oracion con la respuesta correcta.


Su dirección de correo electrónico es FOncativo@hotmail.com


In [ ]:
# TIPS
# Probar primero generando una funcion y llamarla, luego anadir la otra
# Plantearlo paso por paso en distintas celdas, analizar las salidas y las entradas, como identificamos a que funcion llamar?
# luego automatizar dentro de una sola celda


# Lo importante es entregar hasta donde lleguen, sea una funcion, las dos pero sin poder hacer el flujo automatico, lo que puedan, siempre y cuando este
# claro lo que se quizo hacer con comentarios.

## Ejercicio 3

Crear una funcion llamada "history_answer", que toma como parametro de entrada una pregunta sobre un contexto dado y la salida es la respuesta final del proceso impulsado por un LLM.

Dada una historia, el usuario podra hacer preguntas sobre la misma y el LLM debe responder siguiendo los siguientes lineamientos:

REQUISITOS DE LA RESPUESTA
- las respuestas deben ser en base a la historia
- ante la misma pregunta siempre debe responder de la misma manera.
- que responda en solo una oracion.
- el idioma que responde debe ser el mismo que con el que se pregunta (ingles, espanol, portugues).
- que agregue emojis en la oracion que resuman el contenido de la misma.
- que responda siempre en tercera persona.
- si la pregunta no tiene relacion alguna con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
- Responder con 'Hakuna Matata!' al final de **todas** las respuestas (no importa idioma ni cantidad de tokens).

**Ayudin**: 
- No se limiten a usar 1 solo request al LLM, pueden dividirlo en partes para que por un lado se verifique el idioma, por otro lado se verifique si la pregunta tiene relacion con el contexto, etc

- Estructuren bien el prompt procurando separar instrucciones, contexto(historia) y pregunta del usuario.

- Recuerden usar el system message y user message.



In [ ]:
# ejemplo flojo de estructura de prompt
# prompt = f"Responde a la pregunta: {pregunta} de manera concisa y divertida en base a la siguiente historia: {historia}"

In [201]:
import cohere

co = cohere.ClientV2()

historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco."""

def agregar_context(role="",content=""):
    messages.append({"role":role, "content":content})


def history_answer(pregunta):
    # your code here
    #Si la pregunta no tiene nada que ver con el contexto de la historia, debes responder obligatoriamente 'Lo siento no puedo ayudarte con eso.

    sentido = co.chat(
        model="command-r-plus",
        messages = [{"role":"system", "content":f"""Eres un experto en decifrar y puedes interpretar cualquier texto con su sentido mas pracmatico. 
                     Si desifras que la pregunta del usuario no tiene nada que ver con el contexto de la historia {historia}, 
                     responder obligatoriamente 'Lo siento no puedo ayudarte con eso'.
                     Si la pregunta si tiene sentido con el contexto, solo responde que si"""},
                    {"role":"user", "content":f"Tiene sentido la pregunta '{pregunta}' en el contexto de la historia?"}]
    )

    sentido_resp = sentido.message.content[0].text
    #print(sentido_resp)

    traductor = co.chat(
        model="command-r-plus",
        messages = [{"role":"system", "content":"Eres un asistente especializado en traduccion de idiomas. Tu respuesta solo tiene que ser una palabra con el idioma que debe utilizarse"},
                    {"role":"user", "content":f"Segun el idioma en la que se encuentra escrita la pregunta '{pregunta}', debes indicarle al modelo en que idioma debe efectuar su pregunta, para que esten sincronizados los idiomas'"}]
    )

    traductor_resp = traductor.message.content[0].text
    #print(traductor_resp)

    buscador = co.chat(
        model="command-r-plus",
        messages=[{"role":"system","content":f"""Sos un asistente experto en programacion y busqueda. Debes ser capaz de buscar la pregunta en el historial {messages}  del modelo y responder si o no. 
                   .Si tu respuesta a la pregunta es 'Sí', devuélveme una variable 'afirmacion' con el valor 'SI' y otra variable 'respuesta' con la respuesta encontrada en la lista.
                   Si vez varias respuestas sobre esa pregunta, contestame siempre con la ultima. 
                   Si tu respuesta es 'NO', mostrame solamente una variable 'afirmacion' con el valor 'NO'.
                   Responde únicamente con la lista del diccionario en formato Python, sin texto adicional."""},
                  {"role":"user","content":f"La {pregunta} esta repetida en el historial?"}]
    )

    resp_buscador = buscador.message.content[0].text
    #print(resp_buscador)
    

    response_1 = co.chat(
        model="command-r-plus",
        messages = [{"role":"system","content":f"""Eres un profesor de Literatura y Letras y tienes que interpretar el texto: {historia}. 
                     Tienes la capacidad de poder leer cualquier fracmento de texto y poder estudiarlo para tener conocimiento sobre informacion que brinde el texto, 
                     para poder responder cualquier tipo de pregunta hacia el texto. Tu respuesta tiene que ser en una oracion.
                     Debes tener en cuenta la respuesta del modelo '{traductor_resp}' que te idicara con que idioma debes responder. Es obligatorio hacerle caso.
                     Presta atencion a la respuesta del modelo '{sentido_resp}' porque si su respuesta es 'Lo siento, no puedo ayudarte con eso.' debes responder solamente eso obligatoriamente.
                     Debes tener en cuenta, si el modelo anterior de busqueda encuentra la pregunta repetida '{resp_buscador}', debes responder lo que el te indique. Si no esta repetida, se libre de contestar'"""},
                    {"role":"user","content":f"La pregunta del usuario es la siguiente: {pregunta}"}]
    )

    respuesta_model_1 = response_1.message.content[0].text
    #print(respuesta_model_1)

    agregar_context("user",f"{pregunta}")
    agregar_context("assistant",f"{respuesta_model_1}")


  
    emojin = co.chat(
        model = "command-r-plus",
        messages = [{"role":"system", "content":"Eres un chatbot. Debes contemplar el manejo de emojins, que puedas remplazar una palabra con un emojin que signifique o simbolice la palabra que vos elijas"},
                    {"role":"user", "content":f"Elige dos palabras que signifiquen verbos de esta oracion y remplazalas con emojins: {respuesta_model_1}"}]
    )

    respuesta_emojin= emojin.message.content[0].text
    #print(respuesta_emojin)


    tercera_persona = co.chat(
        model = "command-r-plus",
        messages = [{"role":"system", "content":f"""Eres experto en literatura.Tus respuestas se deben configurar siempre en tercera persona. Al final de cada respuesta debes obligatoriamente incluir la frase 'Hakuna Matata! al final de la oracion. Tu respuesta debe ser solamente la oracion en tercera persona.
                     Si la respuesta del modelo '{sentido_resp} es 'Lo siento, no puedo ayudarte con eso.' debes responder solamente eso obligatoriamente y quitando los emojins."""},
                    {"role":"user", "content":f"Convierte esta oracion: {respuesta_emojin} sin la agregar alguna extencion en tu respuesta y sin quitar los emojins"}]
    )

    respuesta_final= tercera_persona.message.content[0].text
    #print(respuesta_final)

    return respuesta_final


pregunta = "Alguien murio en la historia?"


# respuesta
print(history_answer(pregunta))

Sí, un compañero de Thomas cayó 🥵 junto a él en el campo de batalla y 🪦.

Hakuna Matata!


## Ejercicio 4

Crear un chatbot sencillo impulsado por un LLM. 

Dicho bot esta destinado a un usuario final y debe cumplir las siguientes **condiciones en sus respuestas**:

- Responder en no mas de 70 tokens.
- Responder de manera positiva, con un tono entusiasta.
- Responder con consejos útiles, como si fueras un tutor.

 
**Otras consideraciones**:

Respetar el formato de la interfaz provista por el ejercicio.

Ademas agregar al codigo propuesto un historial de conversaciones para que el bot pueda mantener el hilo de lo que se esta hablando. Para probar no usen mas de 3 conversaciones anidadas para no enviarle tantos tokens.

Dejar impreso en el notebook el historial de la conversacion.

In [206]:
import cohere

co = cohere.ClientV2()
chat_history = []

def agregar_context(role="",content=""):
    chat_history.append({"role":role, "content":content})

# Crear widgets de entrada y salida
input_box = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output_box = widgets.Output()



# Función de respuesta simulada del chatbot
def chatbot_response(message):
    # Aquí puedes conectar tu modelo o lógica de chatbot real
    model1 = co.chat(
        model = "command-r-plus",
        messages = [{"role":"system", "content":"""Eres un chatbot que logra mantener conversaciones amigables y preparado para hablar del tema que se trate.
                     Tus respuestas deben ser siempre de forma positiva, muestrate entusiasmado en tu respuesta. Debes responder a la altura de un profesor con experiencia de vida, capaz de dar consejos todo el tiempo.
                     Debes conjugar una oracion que tenga sentido con la charla proporcionada y gramaticalmente bien escrita."""},
                     {"role":"user", "content":f"{message}"}],
        max_tokens=70
    
    )

    respuesta_model1= model1.message.content[0].text

    model2 = co.chat(
        model = "command-r-plus",
        messages = [{"role":"system", "content":"""Comportate como un asistente virtual especializado en reconocer en el texto proporcionado, si logra ser un texto con un formato de respuesta positiva o no.
                     En caso de no ser positiva la respuesta, genera tu una modificacion en su gramatica para que se lea un texto entusiasta.
                     Si la respuesta del modelo es totalmente positiva, toma ese texto y muestrame solamente ese texto"""},
                     {"role":"user", "content":f"{respuesta_model1}"}],
        max_tokens=70
    
    )

    respuesta_model2= model2.message.content[0].text

    agregar_context("user",f"{message}")
    agregar_context("assistant", f"{respuesta_model2}")

    
    

    return respuesta_model2

# Función de manejo del botón
def on_send_button_clicked(b):
    with output_box:
        clear_output(wait=True)
        user_message = input_box.value
        if user_message.strip():
            print(f"Tú: {user_message}")
            response = chatbot_response(user_message)
            print(f"Chatbot: {response}")
        input_box.value = ''

# Asociar función al botón
send_button.on_click(on_send_button_clicked)

# Mostrar widgets
display(input_box, send_button, output_box)
print(chat_history)


Text(value='', placeholder='Escribe tu mensaje aquí...')

Button(description='Enviar', style=ButtonStyle())

Output()

In [ ]:
# print(conversation_history)

### RECOMENDACIONES GENERALES

No se confien probando con un par de respuestas y ya, hagan minimo 5 pruebas por ejercicio para asi tener mas chances de visualizar errores en la generacion del contenido.

Prueben combinar LLMs con programacion convencional para los casos que vean convenientes (decisiones if else, respuestas estaticas, etc)

Prueben con distintos modelos de Cohere, hay algunos optimizados para ciertas aplicaciones.